# Lesson 1: Your first agent with Amazon Bedrock

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


## Start of the lesson

In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-west-2')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [5]:
create_agent_response

{'ResponseMetadata': {'RequestId': '388b1992-e627-4bae-8d51-39cf2300edea',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 11 Aug 2025 14:35:15 GMT',
   'content-type': 'application/json',
   'content-length': '624',
   'connection': 'keep-alive',
   'x-amzn-requestid': '388b1992-e627-4bae-8d51-39cf2300edea',
   'x-amz-apigw-id': 'PJThhFwkvHcEB3g=',
   'x-amzn-trace-id': 'Root=1-6899ffa2-23f1c65149ccc68629ded442'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:975050093618:agent/HRNVLN9QMH',
  'agentId': 'HRNVLN9QMH',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::975050093618:role/c135316a3428926l11177494t1w9750500-BedrockAgentRole-37TIr9Qc8nij',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2025, 8, 11, 14, 35, 15, 313600, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a

In [6]:
agentId = create_agent_response['agent']['agentId']

In [7]:
from helper import *

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [9]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': 'e840e541-c31c-4c5c-9cb6-f58a5f62839e',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 11 Aug 2025 14:35:20 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e840e541-c31c-4c5c-9cb6-f58a5f62839e',
   'x-amz-apigw-id': 'PJTiVEQLvHcEDLQ=',
   'x-amzn-trace-id': 'Root=1-6899ffa8-5055ca1662544d700a8e9f79'},
  'RetryAttempts': 0},
 'agentId': 'HRNVLN9QMH',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 8, 11, 14, 35, 20, 430222, tzinfo=tzlocal())}

In [10]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [12]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [13]:
import uuid

In [14]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [15]:
event_stream = invoke_agent_response["completion"]

In [16]:
for event in event_stream:
    print(event)

{'trace': {'agentAliasId': 'RF8OSM2DDS', 'agentId': 'HRNVLN9QMH', 'agentVersion': '1', 'sessionId': '23c399ee-a835-4089-9223-43152e69ae70', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are an advanced AI agent acting as a front line customer support agent. You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>       ...     </parameters>   </invoke> </function_calls> Here are the functions available: <functions>    </functions> You will ALWAYS follow the below guidelines when you are answering a question: <guidelines> - Think through the user\'s question, extract all data from the questi

{'trace': {'agentAliasId': 'RF8OSM2DDS', 'agentId': 'HRNVLN9QMH', 'agentVersion': '1', 'sessionId': '23c399ee-a835-4089-9223-43152e69ae70', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 856, 'outputTokens': 92}}, 'rawResponse': {'content': '<thinking>I apologize, it seems there was an issue with my previous function call. Let me try a different approach to get the return policy information.</thinking>\n\n<function_calls>\n  <invoke>\n    <tool_name>GetCustomerOrder</tool_name>\n    <parameters>\n      <order_id>123456</order_id>\n    </parameters>'}, 'traceId': '5739635e-74e9-463a-ba65-115c14f335a0-2'}}}}}
{'trace': {'agentAliasId': 'RF8OSM2DDS', 'agentId': 'HRNVLN9QMH', 'agentVersion': '1', 'sessionId': '23c399ee-a835-4089-9223-43152e69ae70', 'trace': {'orchestrationTrace': {'rationale': {'text': 'I apologize, it seems there was an issue with my previous function call. Let me try a different approach to get the return policy informati

In [17]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [18]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 
Agent's thought process:
  Okay, let's go through the steps to handle this return request:

Agent's thought process:
  Apologies, let me try that again in the correct format:

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try a different approach to get the return policy
  information.

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try a different approach to get the return policy
  information.

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  calls. Let me try to get the return policy information in the
  correct format.

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  calls. Let me try to get the return policy information in the
  correct format.

Observation:
  Type: 